In [1]:
# Imports

import pandas as pd
from pandas import Series,DataFrame
import numpy as np

from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

Load titanic train and test csv files

In [3]:
X_train = pd.read_csv("./train.csv")
y_train = X_train.pop("Survived")

X_test_orig = pd.read_csv("./test.csv")

In [4]:
data = X_train.append(X_test_orig, ignore_index=True)

In [5]:
data['Title'] = data.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())
data.Title.value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Dr                8
Rev               8
Col               4
Ms                2
Major             2
Mlle              2
Lady              1
Don               1
Jonkheer          1
Mme               1
Capt              1
Dona              1
Sir               1
the Countess      1
Name: Title, dtype: int64

In [6]:
normalized_titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}
data.Title = data.Title.map(normalized_titles)
data.Title.value_counts()

Mr         757
Miss       262
Mrs        200
Master      61
Officer     23
Royalty      6
Name: Title, dtype: int64

In [7]:
# group by Sex, Pclass, and Title
grouped = data.groupby(['Sex','Pclass', 'Title'])
grouped.Age.median()
data.Age = grouped.Age.apply(lambda x: x.fillna(x.median()))

In [8]:
data['CatAge'] = pd.qcut(data.Age, q=5, labels=False )

In [9]:
data['Family_Size'] = data['Parch'] + data['SibSp'] + 1

In [10]:
# try playing with fare data. I saw people adding values to this field online
data["Fare"].fillna(data.Fare.mean(), inplace=True)
embarked = data.Embarked.value_counts().index[0]
data["Embarked"].fillna(embarked, inplace=True)

In [11]:
data = pd.get_dummies(data, columns=['Sex'], prefix = ['Sex'])

In [12]:
# drop columns won't be useful in analysis and prediction
data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin','Fare'], axis=1)

In [13]:
data.head()

,Pclass,Age,SibSp,Parch,Embarked,Title,CatAge,Family_Size,Sex_female,Sex_male
0,3,22.0,1,0,S,Mr,1,2,0,1
1,1,38.0,1,0,C,Mrs,3,2,1,0
2,3,26.0,0,0,S,Miss,1,1,1,0
3,1,35.0,1,0,S,Mrs,3,2,1,0
4,3,35.0,0,0,S,Mr,3,1,0,1


In [14]:
numeric_variables = list(data.dtypes[data.dtypes != "object"].index)
data = data[numeric_variables]

In [15]:
data.isnull().sum()

Pclass         0
Age            0
SibSp          0
Parch          0
CatAge         0
Family_Size    0
Sex_female     0
Sex_male       0
dtype: int64

In [16]:
X_train = data.iloc[:891]
X_test = data.iloc[891:]

### Modeling

In [17]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [18]:
cross_val_score(dt, X_train, y_train, cv=3, scoring="accuracy")

array([0.76430976, 0.8047138 , 0.77441077])

In [19]:
from sklearn.model_selection import GridSearchCV

dt_params = [
    {
        'max_depth': [1,2,4,8,16,32,64],
        'min_samples_leaf': [1,2,3,4,5,6]
    },
]

dt_cv = GridSearchCV(estimator=dt, param_grid=dt_params, cv=4)

dt_cv = dt_cv.fit(X_train, y_train)

In [20]:
cross_val_score(dt_cv.best_estimator_, X_train, y_train, cv=3, scoring="accuracy")

array([0.76094276, 0.83501684, 0.81481481])

In [21]:
print("Optimal params: {}".format(dt_cv.best_estimator_))

Optimal params: DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=6, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')


<b>Split out test data set</b>

In [22]:
X_train_small = data.iloc[:800]
X_test_small = data.iloc[800:891]
y_train_small = y_train.iloc[:800]
y_test_small = y_train.iloc[800:891]

In [23]:
dt1 = DecisionTreeClassifier(random_state=42)
dt1.fit(X_train_small,y_train_small)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [24]:
cross_val_score(dt1, X_train_small, y_train_small, cv=3, scoring="accuracy")

array([0.71910112, 0.79775281, 0.79323308])

In [25]:
from sklearn.metrics import roc_curve, auc
y_pred_small = dt1.predict(X_test_small)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_small, y_pred_small)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.7739938080495357

In [26]:
from sklearn.metrics import roc_curve, auc
y_pred_small = dt_cv.best_estimator_.predict(X_test_small)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_small, y_pred_small)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.8707430340557275

<b>Exercise 3: Voting Classifier</b>

In [27]:
svc_clf_3 =  SVC(kernel='poly', degree=2)
svc_clf_3.fit(X_train_small, y_train_small)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=2, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [28]:
svc_clf_2 =  SVC()
svc_clf_2.fit(X_train_small, y_train_small)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [29]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier (
    estimators=[('poly_svc', svc_clf_3),
                 ('rbf_svc_2', svc_clf_2),
                 ('best', dt_cv.best_estimator_)],
                 voting='hard')
voting_clf.fit(X_train_small, y_train_small)

VotingClassifier(estimators=[('poly_svc', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=2, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)), ('rbf_svc_2', SVC(C=1.0, cache_size=200, class_...        min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [30]:
cross_val_score(voting_clf, X_train_small, y_train_small, cv=3, scoring="accuracy")

C:\Users\ndbie\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\ndbie\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\ndbie\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([0.80898876, 0.83895131, 0.82330827])

<b>Exercise 4: Random Forest Classifier</b>

In [31]:
rnd_clf2 = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=1)
rnd_clf2.fit(X_train_small, y_train_small)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=16,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [32]:
feat_labels = data.columns
importances = rnd_clf2.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(15):
    print("%2d) %-*s %f" % (f + 1, 20, feat_labels[indices[f]], importances[indices[f]]))

 1) Sex_female           0.325401
 2) Sex_male             0.253968
 3) Pclass               0.142452
 4) Age                  0.112612
 5) Family_Size          0.069080
 6) SibSp                0.037352
 7) CatAge               0.035828
 8) Parch                0.023306


IndexError: index 8 is out of bounds for axis 0 with size 8

<b>Exercise 5: Adaboost</b>

In [33]:
from sklearn.ensemble import AdaBoostClassifier
bdt_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_leaf=2),
                             learning_rate=0.5,
                             n_estimators=3)
bdt_clf.fit(X_train_small, y_train_small)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.5, n_estimators=3, random_state=None)

In [34]:
cross_val_score(bdt_clf, X_train_small, y_train_small, cv=3, scoring="accuracy")

array([0.7752809 , 0.82771536, 0.81578947])

In [35]:
y_pred = dt_cv.predict(X_test)

In [36]:
submission = pd.DataFrame({
        "PassengerId": X_test_orig["PassengerId"],
        "Survived": y_pred
    })
submission.to_csv('titanic.csv', index=False)